# Modelagem de Dados Evolutiva: Sistema de Gerenciamento de Buscas de Trabalho

Vamos estruturar seu projeto em etapas progressivas, partindo da planilha existente (CSV com 800+ aplicações) até um banco de dados relacional robusto para análise e gestão de oportunidades.

## 1. Análise da Base Existente (Etapa Zero)

Antes de modelar, precisamos entender sua estrutura atual:

**Dados atuais no CSV:**
- Quais colunas existem?
- Quais relações já estão implícitas?
- Quais dados estão sendo repetidos desnecessariamente?
- Quais informações importantes estão faltando?

*(Sugiro que você compartilhe um exemplo da estrutura do CSV para podermos mapear melhor)*

### 1.1.Importando os dados

In [35]:
import pandas as pd

df = pd.read_csv('_raw-data/Application Tracker fce54b225d9b4094962ae10c774c15db_all.csv')

# Análise básica
print("=== Estrutura ===")
print(f"Linhas: {len(df)}, Colunas: {len(df.columns)}")
print("\n=== Tipos de Dados ===")
print(df.dtypes)
print("\n=== Dados Faltantes ===")
print(df.isnull().mean().sort_values(ascending=False))

# Análise de colunas críticas
print("\n=== Análise de Empresas ===")
print(df['Company Name'].value_counts().head(10))
print("\n=== Análise de Status ===")
print(df['Status'].value_counts())
df.head(20)

=== Estrutura ===
Linhas: 944, Colunas: 32

=== Tipos de Dados ===
Campaing ID         object
Job Title           object
Date Applied        object
Company Name        object
Status              object
Plataforma RH       object
Modalidade          object
Obs                 object
Insgihts            object
Overview            object
JobDes              object
Industry            object
Company size        object
Headquarters        object
Specialties         object
Job Description     object
Senioridade         object
Vaga Afirmativa     object
Beneficios          object
Cover letter        object
Ageing              object
Company-Data        object
Created             object
GLDR-Index         float64
Milestone          float64
Quarter             object
Rollup              object
Step                object
closed              object
cohort rate        float64
cohort: recusa      object
webarquive          object
dtype: object

=== Dados Faltantes ===
Milestone          1.000000
Co

,Campaing ID,Job Title,Date Applied,Company Name,Status,Plataforma RH,Modalidade,Obs,Insgihts,Overview,...,Created,GLDR-Index,Milestone,Quarter,Rollup,Step,closed,cohort rate,cohort: recusa,webarquive
0,NaN,NaN,NaN,NaN,Convite 3th Entrevista,NaN,NaN,NaN,NaN,NaN,...,"March 25, 2025 11:10 PM",NaN,NaN,NaN,AM Design,NaN,NaN,NaN,NaN,https://web.archive.org/save/
1,NaN,https://www.linkedin.com/posts/kojieumesmo_ple...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"March 20, 2025 10:05 PM",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://web.archive.org/save/
2,NaN,"Olá, Designers! Tenho três oportunidades para ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"March 13, 2025 9:01 PM",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://web.archive.org/save/https:%2F%2Fwww.l...
3,NaN,Developer Front (React) Pleno | CI&T | LinkedIn,"March 11, 2025",CI&T,NaN,NaN,NaN,NaN,NaN,NaN,...,"March 11, 2025 7:12 PM",NaN,NaN,25'Q1,NaN,NaN,NaN,NaN,NaN,https://web.archive.org/save/https:%2F%2Fwww.l...
4,NaN,Pleno UI Design | UX Design,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"March 11, 2025 7:01 PM",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://web.archive.org/save/https:%2F%2Fimpul...
5,NaN,Product Desginer com Inglês Sênior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"February 27, 2025 11:21 AM",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://web.archive.org/save/https:%2F%2Famcom...
6,NaN,Product Designer Especialista,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"February 27, 2025 11:19 AM",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://web.archive.org/save/https:%2F%2Frecla...
7,v24s5230,Product Designer - Sênior | Whirlpool,"December 30, 2024",More Talent Tech,"Aplicado, Currículo, English Test, Inf. Adicio...",recrut.ai,hibrido,NaN,NaN,NaN,...,"February 24, 2025 10:34 AM",NaN,NaN,24'Q4,NaN,99.done,NaN,NaN,NaN,https://web.archive.org/save/
8,v24s5230,Especialista UX,"December 30, 2024",Drogaria Araujo,"13D, 1D, 3D, 5D, Aplicado, Comportamento, Conv...",Easy Aply,"clt, presencial",NaN,NaN,NaN,...,"February 24, 2025 1:19 AM",NaN,NaN,24'Q4,NaN,04.presentation,NaN,NaN,NaN,https://web.archive.org/save/
9,v24s5230,Designer - UI,"December 30, 2024",fueltech,"Aplicado, Currículo, Inf. Adicionais, Inf. Can...",Gupy,hibrido,NaN,NaN,NaN,...,"January 16, 2025 2:03 AM",NaN,NaN,24'Q4,NaN,02.application,NaN,NaN,NaN,https://web.archive.org/save/


### 2.1.Quais colunas existem?
--------------------------

**Colunas principais (bem preenchidas):**

-   `Job Title` (942/944): Título da vaga

-   `Date Applied` (938/944): Data de aplicação (como string)

-   `Company Name` (938/944): Nome da empresa

-   `Status` (938/944): Estado atual da aplicação (valores compostos)

-   `Plataforma RH` (934/944): Plataforma/canal de aplicação

-   `Modalidade` (759/944): Tipo de trabalho (remoto/híbrido/presencial)

-   `Step` (937/944): Estágio do processo seletivo

**Colunas secundárias (pouco preenchidas):**

-   `Senioridade` (9/944): Nível da vaga

-   `JobDes` (9/944) e `Job Description` (14/944): Descrição da vaga

-   `Industry` (4/944): Setor da empresa

-   `Company size` (4/944): Porte da empresa

-   `Cover letter` (8/944): Indica uso de carta de apresentação

**Colunas problemáticas:**

-   `Insgihts` (4/944) e `Overview` (4/944): Dados quase vazios

-   `Obs` (132/944): Observações livres não estruturadas

-   `Milestone` (0/944): Coluna completamente vazia

-   `webarquive` (944/944): Links de archive repetidos

### 2.2\. Quais relações já estão implícitas?
---------------------------------------

**Relações identificáveis:**

1.  **Empresa-Vaga**:

    -   `Company Name` → Cada empresa pode ter múltiplas vagas

    -   Problema: Empresas não normalizadas ("CI&T" vs "CI&T Brasil")

2.  **Vaga-Aplicação**:

    -   `Job Title` + `Company Name` formam identificador implícito

    -   Falta: ID único para vagas

3.  **Status-Processo**:

    -   `Status` contém múltiplos estados separados por vírgula

    -   `Step` parece ser um estágio numérico do processo

4.  **Canal-Aplicação**:

    -   `Plataforma RH` indica origem da vaga (LinkedIn, site da empresa etc.)

### 2.3\. Quais dados estão sendo repetidos desnecessariamente?
---------------------------------------------------------

**Principais redundâncias:**

1.  **Informações de Empresa**:

    -   Mesmo `Company Name` repetido para cada aplicação

    -   Dados como `Industry`, `Company size` duplicados (quando preenchidos)

2.  **Dados de Status**:

    -   Histórico armazenado como string composta na coluna `Status`

    -   Exemplo: "Aplicado, Currículo, Inf. Adicionais, Inf. Cancelada"

3.  **Webarchive**:

    -   Mesmo link repetido em todas as linhas sem necessidade

4.  **Job Title vs JobDes**:

    -   Duplicação de informação sobre título da vaga

### 2.4\. Quais informações importantes estão faltando?
-------------------------------------------------

**Dados ausentes críticos:**

1.  **Metadados de Aplicação**:

    -   Data/hora exata de cada mudança de status

    -   Feedback estruturado de recrutadores

2.  **Classificação de Vagas**:

    -   Área específica (UX, Frontend, etc.)

    -   Tecnologias/tags relevantes

    -   Faixa salarial

3.  **Informações de Processo**:

    -   Tempo entre etapas

    -   Contato específico em cada empresa

    -   Materiais enviados (versão do currículo)

4.  **Dados de Empresa Enriquecidos**:

    -   Site corporativo

    -   Cultura/benefícios

    -   Notas pessoais sobre a empresa

### 2.5\. Mapa de Transformação Recomendado
-------------------------------------

| Problema Atual | Solução Proposta |
| --- |  --- |
| Valores compostos em `Status` | Tabela `historico_status` com relacionamento 1-N |
| --- |  --- |
| Empresas não normalizadas | Tabela `empresa` com ID único |
| Múltiplas colunas para descrição | Unificar em `descricao_vaga` |
| Datas como strings | Converter para tipo `date` |
| Modalidade inconsistente | Tabela `tipo_trabalho` com valores padronizados |
| Falta histórico temporal | Registrar timestamps de cada mudança |

6\. Próximos Passos Imediatos
-----------------------------